In [1]:
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
import seaborn as sns

from statsmodels.stats.multitest import fdrcorrection,multipletests

# Set the default font to Arial
import matplotlib
matplotlib.rcParams['font.family'] = 'Arial'

# Parameters

In [2]:
# The filtering based on hotspots.
filter_version = 'including'  # 'including', 'excluding', 'only'
list_filter_versions = ['including', 'excluding', 'only']

PATH = '../../results/chrM_2024/'

INDEL_PREFIX = {
        'including':'indels/indels.denovo_', 
        'excluding':'indels/indels.exclHotspots.denovo_', 
        'only':'indels/indels.onlyHotspots.denovo_' 
    }[filter_version]

FREQS_PREFIX = {
        'including':'tables/freqs.denovo_indels.', 
        'excluding':'tables/freqs.exclHotspots.', 
        'only':'tables/freqs.onlyHotspots.' 
    }[filter_version]


# The 3 species.
list_species = ['mouse','macaque','human']
# The alignment chosen.
alignment = 'chrM'
# Dictionary of tissues per species.
dict_tissues =  {
                    'mouse':['Oo', 'M', 'Br'], 
                    'macaque':['Oo', 'M', 'Li'],
                    'human':['Oo', 'Sa', 'Bl'],
                }
dict_tissue_names = {'Oo':'Oocytes','M':'Skeletal muscle','Br':'Brain',
                'Li':'Liver','Sa':'Saliva','Bl':'Blood'}

# Wrangle CDS annotations

In [ ]:
import pandas as pd
from Bio import SeqIO

dict_files = {'macaque':'sequence.rhesus_macaque.NC_005943.gb', 
			  'mouse':'sequence.mouse.NC_005089.gb', 
			  'human':'sequence.human.NC_012920.gb'}

for species in dict_files:

	# Parse the GenBank file using SeqIO
	gb_file = dict_files[species]
	record = SeqIO.read(gb_file, 'genbank')

	# Extract key information
	print(f"Sequence ID: {record.id}")
	print(f"Description: {record.description}")
	print(f"Sequence length: {len(record.seq)}")
	print(f"Number of features: {len(record.features)}")

	# Display features
	# for feature in record.features:
		# print(f"Feature type: {feature.type}, Location: {feature.location}")

	# Filter CDS features
	cds_features = [feature for feature in record.features if feature.type == 'CDS']
	print(f"\nNumber of CDS features: {len(cds_features)}")

	# Display CDS features
	for cds in cds_features:
		print(f"CDS Location: {cds.location}, Qualifiers: {cds.qualifiers}")

		# Convert CDS features to a pandas DataFrame
		cds_data = []
		for cds in cds_features:
			cds_data.append({
				'Location': str(cds.location),
				'Start': cds.location.start,
				'End': cds.location.end,
				'Strand': cds.location.strand,
				'Qualifiers': cds.qualifiers
			})

		cds_df = pd.DataFrame(cds_data)
		cds_df

	# Extract coordinates as a list in BED format
	coordinates = cds_df[['Start', 'End', 'Strand']].values.tolist()
	# Use 0-based start positions
	coordinates = [[c[0] - 1, c[1], c[2]] for c in coordinates]
	coordinates

	# Write coordinates to BED file
	out_file = gb_file.replace('.gb', '.cds.bed')
	with open(out_file, 'w') as bed_file:
		for i, coord in enumerate(coordinates):
			chrom = record.id
			start = coord[0]
			end = coord[1]
			strand = '+' if coord[2] == 1 else '-'
			bed_file.write(f"{chrom}\t{start}\t{end}\t.\t.\t{strand}\n")

	print(f"\nBED file created: {out_file}\n")

	with open(out_file, 'r') as f:
		print(f.read())

# Merge overlapping CDS regions using bedtools.
!bedtools merge -i sequence.human.NC_012920.cds.bed > sequence.human.NC_012920.cds.merged.bed
!bedtools merge -i sequence.mouse.NC_005089.cds.bed > sequence.mouse.NC_005089.cds.merged.bed
!bedtools merge -i sequence.rhesus_macaque.NC_005943.cds.bed > sequence.rhesus_macaque.NC_005943.cds.merged.bed

# Import CDS annotations

In [3]:
import pandas as pd

def get_cds():
	dict_files = {'macaque':'sequence.rhesus_macaque.NC_005943.cds.merged.bed', 
			  			'mouse':'sequence.mouse.NC_005089.cds.9821insA.merged.bed', 
			  			'human':'sequence.human.NC_012920.cds.merged.bed'}
	dict_genome_sizes = {'macaque':16564, 'mouse':16300, 'human':16569}
	for species in dict_files.keys():
		cds_file = dict_files[species]
		df_cds = pd.read_csv(cds_file, sep='\t', header=None, names=['chrom','start','end'])
		df_cds['species'] = species
	df_merged = pd.concat( [pd.read_csv(dict_files[sp], sep='\t', header=None, names=['chrom','start','end']).assign(species=sp) for sp in dict_files.keys()] )
	df_merged['genome_size'] = df_merged['species'].map(dict_genome_sizes)
	df_merged['cds_length'] = df_merged['end'] - df_merged['start']
	df_cds = df_merged[['species','start','end']]
	
	return df_cds

df_cds = get_cds()
df_cds		

,species,start,end
0,macaque,3257,4213
1,macaque,4419,5462
2,macaque,5848,7418
3,macaque,7530,8215
4,macaque,8355,9981
5,macaque,10048,10395
6,macaque,10459,12128
7,macaque,12326,14667
8,macaque,14739,15881
0,mouse,2749,3707


# Import indels for 3 species

In [4]:
def get_indels(filter_version):
    # Combine indels for the three species.
    INDEL_PREFIX = {
        'including':'indels/indels.denovo_', 
        'excluding':'indels/indels.exclHotspots.denovo_', 
        'only':'indels/indels.onlyHotspots.denovo_' 
    }
    list_df = []
    for species in list_species:
        df = pd.read_table( f'{PATH+INDEL_PREFIX[filter_version]}{alignment}_{species}.tab' )
        list_df.append( df )
    df_indels = pd.concat(list_df)
    df_indels['Indel_length'] = abs(df_indels['REF'].str.len() - df_indels['ALT'].str.len())
    df_indels['Indel_type'] = np.where( df_indels['REF'].str.len() < df_indels['ALT'].str.len(), 'insertion', 'deletion' )  
    
    # Annotate as CDS or non-CDS.
    df_cds = get_cds()
    def is_in_cds(row):
        species = row['Species']
        pos = row['POS']
        df_cds_sp = df_cds[ df_cds['species'] == species ]
        for _, cds_row in df_cds_sp.iterrows():
            if cds_row['start'] <= pos < cds_row['end']:
                return True
        return False
    df_indels['is_CDS'] = df_indels.apply(is_in_cds, axis=1)
    
    # Only CDS indels
    df_indels = df_indels[ df_indels['is_CDS'] ]

    return df_indels

df_indels = get_indels('including')
df_indels

,CHROM,POS,REF,ALT,INFO,SRR_ID,value,Species,AC,AF,...,Tissue_type,Classification,Mutation_ID,Pedigree_shared,Filter_ID,Region,Region_size,Indel_length,Indel_type,is_CDS
0,chrM,12343,AT,A,SB=2.42169;AF=0.00495049;AC=0,SRR10068699,.:1:0.00495049:2.42169,mouse,1.0,0.004950,...,germline,Germline de novo,12343_AT_A,Unique to Individual,G133p4_12343,non-D-loop,15423,1,deletion,True
1,chrM,13053,T,TC,SB=2.11058;AF=0.00329761;AC=0,SRR10068701,.:2:0.00115607:2.64192,mouse,2.0,0.001156,...,germline,Germline de novo,13053_T_TC,Between pedigrees,G133p4_13053,non-D-loop,15423,1,insertion,True
2,chrM,14806,CACCCCTACTATACAATCAAAGATATCCTAGGTATCCTAATCATAT...,C,SB=2.57858;AF=0.000629327;AC=0,SRR10068701,.:1:0.000629327:2.57858,mouse,1.0,0.000629,...,germline,Germline de novo,14806_CACCCCTACTATACAATCAAAGATATCCTAGGTATCCTAA...,Unique to Individual,G133p4_14806,non-D-loop,15423,58,deletion,True
3,chrM,14640,A,AG,SB=1.61939;AF=0.00191388;AC=0,SRR10068702,.:2:0.00191388:1.61939,mouse,2.0,0.001914,...,somatic,Somatic de novo,14640_A_AG,Unique to Individual,G133p4_14640,non-D-loop,15423,1,insertion,True
5,chrM,13053,T,TC,SB=2.11058;AF=0.00329761;AC=0,SRR10068708,.:2:0.00754717:2.07087,mouse,2.0,0.007547,...,somatic,Somatic de novo,13053_T_TC,Between pedigrees,G131p1_13053,non-D-loop,15423,1,insertion,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,chrM,11030,GA,G,SB=1.51818;AF=0.00198807;AC=0,hs023_Sa,.:1:0.00198807:1.41855,human,1.0,0.001988,...,somatic,Somatic de novo,11030_GA_G,NaN,hs023_11030,non-D-loop,15447,1,deletion,True
131,chrM,11865,A,AC,SB=1.80374;AF=0.00170358;AC=0,hs023_Sa,.:1:0.00170358:1.80339,human,1.0,0.001704,...,somatic,Somatic de novo,11865_A_AC,NaN,hs023_11865,non-D-loop,15447,1,insertion,True
132,chrM,12383,T,TC,SB=1.91172;AF=0.0018315;AC=0,hs023_Sa,.:1:0.0018315:1.91482,human,1.0,0.001832,...,somatic,Somatic de novo,12383_T_TC,NaN,hs023_12383,non-D-loop,15447,1,insertion,True
133,chrM,13131,CT,C,SB=1.79468;AF=0.00168067;AC=0,hs023_Sa,.:1:0.00168067:1.79468,human,1.0,0.001681,...,somatic,Somatic de novo,13131_CT_C,NaN,hs023_13131,non-D-loop,15447,1,deletion,True


# Table indel lengths by species

In [5]:
table_lengths = pd.DataFrame(df_indels.groupby(['Species','Indel_type','Indel_length']).size()).reset_index().rename(columns={0:'Count'})
# enforce desired species order (mouse, macaque, human) then sort
table_lengths['Species'] = pd.Categorical(table_lengths['Species'], categories=list_species, ordered=True)
table_lengths = table_lengths.sort_values(['Species','Indel_type','Indel_length']).reset_index(drop=True)
table_lengths['mult3'] = (table_lengths['Indel_length'] % 3 == 0)
table_lengths['mult3'] = table_lengths['mult3'].map({True: 'mult3', False: '.'}).astype('category')
table_lengths = table_lengths[['Species', 'Indel_type', 'Indel_length', 'mult3', 'Count']]
table_lengths.to_csv(f'indel_lengths_by_species.{filter_version}.cds.tsv', sep='\t', index=False)
table_lengths

,Species,Indel_type,Indel_length,mult3,Count
0,mouse,deletion,1,.,133
1,mouse,deletion,2,.,18
2,mouse,deletion,3,mult3,15
3,mouse,deletion,4,.,1
4,mouse,deletion,5,.,5
...,...,...,...,...,...
94,human,insertion,1,.,23
95,human,insertion,3,mult3,1
96,human,insertion,6,mult3,1
97,human,insertion,7,.,1


# Import CDS indel counts

In [6]:
import pandas as pd
tab = pd.read_table("indel_lengths_by_species.including.cds.tsv")
tab

,Species,Indel_type,Indel_length,mult3,Count
0,mouse,deletion,1,.,133
1,mouse,deletion,2,.,18
2,mouse,deletion,3,mult3,15
3,mouse,deletion,4,.,1
4,mouse,deletion,5,.,5
...,...,...,...,...,...
94,human,insertion,1,.,23
95,human,insertion,3,mult3,1
96,human,insertion,6,mult3,1
97,human,insertion,7,.,1


# Proportion z-test

In [7]:
from scipy.stats import norm

# Recalculate results using one-sample proportion z-test
results = []

for species in tab['Species'].unique():
	for indel_type in tab['Indel_type'].unique():
		subset = tab[(tab['Species'] == species) & (tab['Indel_type'] == indel_type)]
		
		# Create contingency table: mult3 vs non-mult3
		mult3_count = subset[subset['mult3'] == 'mult3']['Count'].sum()
		non_mult3_count = subset[subset['mult3'] == '.']['Count'].sum()
		
		if mult3_count + non_mult3_count > 0:
			total = mult3_count + non_mult3_count
			p_observed = mult3_count / total
			p_null = 1/3  # Expected proportion under null hypothesis
			
			# One-sample proportion z-test
			se = (p_null * (1 - p_null) / total) ** 0.5
			z_stat = (p_observed - p_null) / se
			p_value = 2 * (1 - norm.cdf(abs(z_stat)))  # Two-tailed test
			
			results.append({
				'Species': species,
				'Indel_type': indel_type,
				'Mult3_count': mult3_count,
				'Non_mult3_count': non_mult3_count,
				'Mult3_proportion': p_observed,
				'z_statistic': z_stat,
				'p_value': p_value
			})

results_df = pd.DataFrame(results)
# Add significance column based on p-value thresholds
results_df['Significance'] = results_df['p_value'].apply(
	lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
)
# Add enrichment direction column
results_df['Direction'] = results_df['Mult3_proportion'].apply(
	lambda x: 'Enriched' if x > 1/3 else 'Depleted'
)
results_df['Species'] = results_df['Species'].str.capitalize()
results_df['Indel_type'] = results_df['Indel_type'].str.capitalize()
results_df.to_csv("enrichment_of_multiples_of_3_CDS_recalculated.tsv", sep="\t", index=False)
results_df

,Species,Indel_type,Mult3_count,Non_mult3_count,Mult3_proportion,z_statistic,p_value,Significance,Direction
0,Mouse,Deletion,20,165,0.108108,-6.498441,8.115686e-11,***,Depleted
1,Mouse,Insertion,2,58,0.033333,-4.929503,8.243906e-07,***,Depleted
2,Macaque,Deletion,135,819,0.141509,-12.568492,0.000000e+00,***,Depleted
3,Macaque,Insertion,16,349,0.043836,-11.732696,0.000000e+00,***,Depleted
4,Human,Deletion,5,34,0.128205,-2.717465,6.578414e-03,**,Depleted
5,Human,Insertion,2,25,0.074074,-2.857738,4.266725e-03,**,Depleted
